<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/MemProtMD_Association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/pstansfeld/MemProtMD/raw/main/mr-membrane-protein.png" height="200" align="right" style="height:240px">

##MemProtMD Association

[MemProtMD](https://doi.org/10.1016/j.str.2015.05.006) membrane protein association protocol for allowing a protein to self-associate with a preformed lipid membrane. 

The protein structure is converted to [Martini 3](https://www.nature.com/articles/s41592-021-01098-3) coarse-grained (CG) representation using [Martinize](https://github.com/marrink-lab/vermouth-martinize) with a lipid membrane constructed using [Insane](https://doi.org/10.1021/acs.jctc.5b00209). The system is briefly equilibrated using [GROMACS](https://doi.org/10.1016/j.softx.2015.06.001).

Inspiration for this Google Colab was taken from the amazing work of the [ColabFold](https://github.com/sokrypton/ColabFold) team.

Change the settings below and click **Runtime → Run all**.  You will be prompted to upload a Soluble Protein PDB file to simulate.

In [ ]:
#@title Define Settings
#@markdown ---
#@markdown #### Set Membrane Type:
MembraneType = "POPE:POPG" #@param ["POPC","POPE:POPG","POPE:POPG:CARDIOLIPIN"]
#@markdown ---
#@markdown #### Set number of Repeats
Repeats = 5 #@param {type:"slider", min:1, max:25, step:1}
#@markdown ---
#@markdown #### Define Distance between Protein and Membrane:
Distance = 10 #@param {type:"slider", min:0, max:25, step:1}
#@markdown ---
#@markdown #### Set Box Dimensions:
Box_Width = 12 #@param {type:"slider", min:8, max:30, step:1}
Box_Length = 25 #@param {type:"slider", min:12, max:50, step:1}
#@markdown ---

In [ ]:
#@title Upload PDB coordinate files
import os
import sys

cube = [Box_Width,Box_Width,Box_Length]

if MembraneType == "POPC":
  lipid = '-l POPC:1'
elif MembraneType == "POPE:POPG":
  lipid = '-l POPE:7 -l POPG:3'
elif MembraneType == "POPE:POPG:CARDIOLIPIN":
  lipid = '-l POPE:7 -l POPG:2 -l CARD:1'

!python3 -m pip install py3dmol
!python3 -m pip install colorama

import py3Dmol
from colorama import Fore
from google.colab import files
sys.path.append('/usr/local/lib/python3.7/site-packages/')

os.chdir('/content/')
print(Fore.BLUE + "\nUpload Protein Structure:\n")
upload = files.upload()
mview = py3Dmol.view(width=800,height=400)  
filename = next(iter(upload))
name = os.path.splitext(filename)[0]
working_dir = '/content/' + name +'/'
os.makedirs(working_dir, exist_ok=True)
os.rename(filename, working_dir + filename)
os.chdir(working_dir)
mol1 = open(working_dir + filename, 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

In [ ]:
#@title Install dependencies
%%capture
os.chdir('/content/')
if not os.path.isdir("/content/cg2at/"):
  !apt-get update -y
  !apt-get install dssp
  !python3 -m pip install pdb2pqr
  !python3 -m pip install vermouth
  !python3 -m pip install GromacsWrapper
  !python3 -m pip install MDAnalysis 
  !git clone https://github.com/pstansfeld/cg2at
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/martini_v300.zip
  !unzip -o martini_v300.zip
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/insane3.py
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/gromacs.zip
  !unzip -o gromacs.zip
  !ln -s /content/content/gromacs/bin/gmx /usr/bin/gmx

import gromacs
import MDAnalysis
import glob
import shutil
from random import randrange

In [ ]:
#@title Set-up Coarse-Grained Membrane Protein Systems
%%capture
os.chdir(working_dir)

for file in glob.glob(r'/content/martini*.itp'):
    print(file)
    shutil.copy(file, working_dir)

gromacs.make_ndx(f=filename,o='index.ndx',input=('del 0', 'del 1-100','q'))
gromacs.editconf(f=filename,o='protein.pdb',n='index.ndx',c=True,box=cube,input=(0,0),backup=False,label='A',resnr=1)

u = MDAnalysis.Universe('protein.pdb')
x = round(u.dimensions[0]/10)
y = round(u.dimensions[1]/10)
z = round(u.dimensions[2]/10)

with open('protein.pdb', 'r') as file :
  filedata = file.read()
filedata = filedata.replace('HSE', 'HIS')
filedata = filedata.replace('HSD', 'HIS')
filedata = filedata.replace('MSE', 'MET')
filedata = filedata.replace(' SE ', ' SD ')
with open('protein.pdb', 'w') as file:
  file.write(filedata)
with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001')
with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein-cg.itp"\n[ system ]\n[ molecules ]\nProteinA 1')

!martinize2 -f protein.pdb -dssp mkdssp -ff martini3001 -x protein-cg.pdb -o protein-cg.top -elastic -ef 500 -eu 0.7 -el 0.5 -ea 0 -ep 0 -merge A -maxwarn 100000

!sed -e 's/^molecule.*/ProteinA 1/g' molecule_0.itp >  protein-cg.itp
gromacs.editconf(f='protein-cg.pdb',o='protein-cg.pdb',rotate=[randrange(360),randrange(360),0])
gromacs.editconf(f='protein-cg.pdb', o='protein-box.pdb', c=True, box=cube)
gromacs.grompp(f='em.mdp',o='em.tpr',c='protein-box.pdb',maxwarn='-1',backup=False,v=True)
gromacs.mdrun(deffnm='em', c='protein-em.pdb',backup=False)

for rep in range(1,Repeats+1):
  rep=str(rep)
  os.chdir(working_dir)
  os.system('python2.7 /content/insane3.py '+ lipid +' -salt 0.15 -sol W -o CG-system.gro -p topol.top -f protein-em.pdb -center -rotate random -ring -x %s -y %s -z %s -dm %s' % (x, y, z, Distance))

  replacements = {'Protein        1\n':'ProteinA       1\n','NA+':'NA', 'CL-':'CL', '#include "martini_v3.itp"':'#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n'}
  lines = []
  with open('topol.top') as infile:
      for line in infile:
          for src, target in replacements.items():
              line = line.replace(src, target)
          lines.append(line)
  with open('topol.top', 'w') as outfile:
      for line in lines:
          outfile.write(line)

  os.system('cp topol.top topol'+rep+'.top')

  gromacs.grompp(f='em.mdp',o='em'+ rep +'.tpr',c='CG-system.gro',maxwarn='-1',backup=False,v=True)
  gromacs.mdrun(deffnm='em'+ rep, c='CG-system'+ rep +'.pdb',backup=False)
  gromacs.trjconv(f='CG-system'+ rep +'.pdb', o='CG-system'+ rep +'.pdb', pbc='mol', s='em'+ rep +'.tpr', conect=True, input='0',backup=False)

  os.makedirs('EQ'+rep, exist_ok=True)
  gromacs.make_ndx(f='CG-system'+ rep +'.pdb', o='index'+ rep +'.ndx', input=('del 0', 'del 1-40', '0|rPOP*','1&!0','!1','del 1','name 1 Lipid','name 2 SOL_ION','q'),backup=False)
  with open('cgequil.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 10000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\nref_t  = 310 310 310\nPcoupl  = berendsen\nPcoupltype  = semiisotropic\ntau_p  = 12.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\nlincs_warnangle = 30\n')
  gromacs.grompp(f='cgequil.mdp',o='EQ'+ rep +'/eq',c='CG-system'+ rep +'.pdb',maxwarn=-1, n='index'+ rep +'.ndx',backup=False)  
  os.chdir(working_dir + '/EQ' + rep)
  gromacs.mdrun(deffnm='eq',backup=False, v=True)
  os.chdir(working_dir)
  os.makedirs('MD'+rep, exist_ok=True)
  with open('cgmd.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 250000000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\nref_t  = 310 310 310\nPcoupl  = berendsen\nPcoupltype  = semiisotropic\ntau_p  = 12.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\nlincs_warnangle = 30\n')
  gromacs.grompp(f='cgmd.mdp',o='MD'+ rep +'/md',c='EQ'+ rep +'/eq.gro',maxwarn=-1, n='index'+ rep +'.ndx',backup=False)  
  

In [ ]:
#@title Visualise Final systems
for rep in range(1,Repeats+1):
  print(Fore.BLUE + '\nRepeat ' + str(rep) + '\n')
  rep=str(rep)
  mview = py3Dmol.view(width=800,height=400)  
  mol1 = open(working_dir + 'EQ'+rep +'/eq.gro', 'r').read()
  mview.addModel(mol1,'gro')
  mview.setStyle({'cartoon':{'color':'spectrum'}})
  mview.setStyle({'atom':'PO1','atom':'PO2','atom':'PO3','atom':'PO4'},{'sphere':{}})
  mview.setStyle({'atom':'BB'},{'sphere':{}})
  mview.setBackgroundColor('0xffffff')
  mview.zoomTo()
  mview.show()
  for file in glob.glob(r'#*'):
      os.remove(file)
    
os.chdir(working_dir)
for file in glob.glob(r'#*'):
    os.remove(file)

In [ ]:
#@title Download Zip
os.chdir(working_dir)
shutil.rmtree('MD', ignore_errors=True)
try:
    os.remove('em*.trr')
except OSError:
    pass
try:
    os.remove('em*.tpr')
except OSError:
    pass
try:
    os.remove('em*.log')
except OSError:
    pass
try:
    os.remove('protein.pdb')
except OSError:
    pass
try:
    os.remove('molecule_0.itp')
except OSError:
    pass
try:
    os.remove('mdout.mdp')
except OSError:
    pass
try:
    os.remove('aligned.gro')
except OSError:
    pass
try:
    os.remove('protein-cg.top')
except OSError:
    pass
os.chdir('/content/')
os.system('zip -r ' + name + '.zip ' + name )
files.download(name + '.zip')